# Xboost Model

## Load libraries

In [ ]:
# Importar librerías que vamos a estar usando en general 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm, trange
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

# Importar funcions per evaluar els models
from utils import evaluate_models
from utils import plot_model_evaluation

## Load data

In [ ]:
pth = "C:/Users/34627/Documents/GitHub/Gorrapiedra-proyecto/data/formated data"

df_20 = pd.read_csv(os.path.join(pth, "data_2020.csv"), index_col=False, skipinitialspace=True, skip_blank_lines=True)
df_21 = pd.read_csv(os.path.join(pth, "data_2021.csv"), index_col=False, skipinitialspace=True, skip_blank_lines=True)
df_22 = pd.read_csv(os.path.join(pth, "data_2022.csv"), index_col=False, skipinitialspace=True, skip_blank_lines=True)
df_23 = pd.read_csv(os.path.join(pth, "data_2023.csv"), index_col=False, skipinitialspace=True, skip_blank_lines=True)
df = pd.concat([df_20, df_21, df_22, df_23], ignore_index = True)

if 'Unnamed: 0' in df.columns:
    df = df.drop(['Unnamed: 0'], axis=1)   # Eliminar columna Unnamed

# Visualitzar el df
df.head()

In [ ]:
# Separar les característiques (X) de la variable objectiu (y)
X = df.drop(['percentage_docks_available'], axis=1)
y = df['percentage_docks_available']

In [ ]:
# Dividim les dades en conjunts de training i testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Model XGBOOST

In [ ]:
# Creem model de XGBoost Regression
model_xgbr = XGBRegressor()

# Entrenem el model
model_xgbr.fit(X_train, y_train)

# Fem les prediccions
y_pred_xgbr = model_xgbr.predict(X_test)

In [ ]:
# Evaluem el model
results = evaluate_models(y_test, y_pred_xgbr)
print(results)

In [ ]:
# Evaluem el model
plot_model_evaluation(y_test, y_pred_xgbr)